In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('white')
%matplotlib inline

In [17]:
# column_names = ['otel_name', 'otel_namex', 'rate', 'ratex','review','price','adress']
df = pd.read_csv('C:\\Users\\kbc\\Downloads\\hotel.csv')
df.drop('Unnamed: 0', axis=1, inplace=True)
df.head()

,title,rating,review,address,user_id
0,Mövenpick Hotel Ankara,2.7,1,"Beştepeler Mah. Yaşam Cad. No: 1 Söğütözü, Ank...",3
1,Atalay Hotel,2.9,52,"Çankırı Cad. No 20, Ankara 06030 Türkiye",1
2,Hotel Ergen,1.9,6,"Karanfil Sok. No:48 Kızılay, Çankaya, Ankara 0...",2
3,Hotel Ayma,2.1,13,"Meşrutiyet Cad. No:25 Kızılay, Çankaya, Ankara...",2
4,Erk Otel,2.4,1,"Menekşe Sok. No:27 Kızılay, Çankaya, Ankara Tü...",6


In [18]:
df.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
title,2750,320,Limak Ambassadore Hotel,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rating,2750,NaN,NaN,NaN,2.32364,1.11939,0,1.6,2.4,3.1,5
review,2750,NaN,NaN,NaN,53.8771,110.417,1,4,18,53,970
address,2750,320,"Mithat Paşa Cad. No:4 Sıhhıye, Ankara 06410 Tü...",10,NaN,NaN,NaN,NaN,NaN,NaN,NaN
user_id,2750,NaN,NaN,NaN,5.5,2.88468,1,3,6,8,10


In [5]:
a = pd.DataFrame(df.groupby('title')['rating'].mean().sort_values(ascending=False))
a['num of ratings'] = pd.DataFrame(df.groupby('title')['rating'].count())
a.head()

,rating,num of ratings
title,,
Sc Inn Hotel Ankara,3.483333,6
Business Park Hotel,3.433333,6
Guleryuz Hotel,3.262500,8
Reda Palas,3.250000,6
New Park Hotel,3.230000,10


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
rating,2750.0,2.323636,1.119389,0.0,1.6,2.4,3.1,5.0
review,2750.0,53.877091,110.417269,1.0,4.0,18.0,53.0,970.0
user_id,2750.0,5.500000,2.884682,1.0,3.0,6.0,8.0,10.0


In [7]:
matrix_hotel = df.pivot_table(index='user_id', columns='title', values='rating')
matrix_hotel.head()

title,A Apart Otel,A&C Kiz Ogrenci Yurdu 3. Sube,Ab-i Hayat Thermal Hotel,Ada Palas Otel,Aden Apart,Ak Life Hotel & Suite,Akar International Hotel,Akman TZOB Otel,Aktaş Otel,Akya Hotel,...,Yeni Yavuz Hoteli,Yildiz Hotel,Yurt Akademi Ankara,Zarina Butik Hotel,concept house,Çankaya İmkb Mesleki ve Teknik Anadolu Lisesi Uygulama Oteli,Çankırı Büyük Hotel,Ümit Pembe Köşk Hotel,İlci Residence Otel,Şahinbey Otel
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.6,NaN,2.3,1.6,3.0,2.8,2.6,0.5,1.9,...,2.4,NaN,NaN,2.1,2.8,1.2,3.6,1.8,3.5,2.0
2,NaN,1.4,NaN,2.6,NaN,4.1,2.6,3.5,5.0,1.6,...,2.8,1.8,NaN,3.0,0.0,NaN,NaN,2.7,3.3,2.1
3,1.8,2.8,3.5,4.5,1.7,3.1,4.3,3.3,1.4,0.3,...,3.0,1.5,1.4,NaN,NaN,2.8,2.4,3.3,1.0,NaN
4,4.3,2.1,1.3,NaN,1.9,2.8,2.3,3.0,1.9,1.4,...,1.0,4.5,3.1,1.3,2.0,0.4,2.8,1.1,2.3,0.0
5,NaN,1.5,1.4,2.1,3.0,2.8,1.3,2.3,3.3,NaN,...,1.2,3.1,1.3,NaN,3.6,NaN,2.8,1.6,1.7,2.5


In [9]:
a.sort_values('rating',ascending=False).head(10)

,rating,num of ratings
title,,
Sc Inn Hotel Ankara,3.483333,6
Business Park Hotel,3.433333,6
Guleryuz Hotel,3.262500,8
Reda Palas,3.250000,6
New Park Hotel,3.230000,10
Ankara Trend House Apart,3.214286,7
Koyunlu Hotel,3.200000,8
Park Otel,3.157143,7
Patalya Lakeside Resort Hotel,3.114286,7


In [10]:
Elit_Otel_user_ratings = matrix_hotel['Elit Otel']
Ankara_HiltonSA_user_ratings = matrix_hotel['Ankara HiltonSA']
Elit_Otel_user_ratings.head()


user_id
1    2.0
2    NaN
3    0.5
4    3.5
5    2.5
Name: Elit Otel, dtype: float64

In [11]:
Ankara_HiltonSA_user_ratings.head()


user_id
1    0.0
2    1.8
3    2.6
4    1.8
5    3.1
Name: Ankara HiltonSA, dtype: float64

In [13]:
similar_to_elite = matrix_hotel.corrwith(Elit_Otel_user_ratings, method='pearson')
similar_to_hilton = matrix_hotel.corrwith(Ankara_HiltonSA_user_ratings, method='pearson')
similar_to_elite

title
A Apart Otel                                                    0.456346
A&C Kiz Ogrenci Yurdu 3. Sube                                  -0.662580
Ab-i Hayat Thermal Hotel                                       -0.767142
Ada Palas Otel                                                 -0.379068
Aden Apart                                                      0.009744
                                                                  ...   
Çankaya İmkb Mesleki ve Teknik Anadolu Lisesi Uygulama Oteli    0.029309
Çankırı Büyük Hotel                                            -0.512447
Ümit Pembe Köşk Hotel                                          -0.730659
İlci Residence Otel                                             0.222623
Şahinbey Otel                                                  -0.303094
Length: 320, dtype: float64

In [14]:
similar_to_hilton


title
A Apart Otel                                                   -0.388925
A&C Kiz Ogrenci Yurdu 3. Sube                                  -0.088885
Ab-i Hayat Thermal Hotel                                       -0.105623
Ada Palas Otel                                                  0.325931
Aden Apart                                                      0.571719
                                                                  ...   
Çankaya İmkb Mesleki ve Teknik Anadolu Lisesi Uygulama Oteli    0.364638
Çankırı Büyük Hotel                                            -0.156307
Ümit Pembe Köşk Hotel                                          -0.264289
İlci Residence Otel                                            -0.880788
Şahinbey Otel                                                  -0.011440
Length: 320, dtype: float64

In [16]:
corr_hilton = pd.DataFrame(similar_to_hilton,columns=['Correlation'])
corr_hilton.dropna(inplace=True)
corr_hilton = corr_hilton.join(a['num of ratings'])
corr_hilton.sort_values('Correlation',ascending=False).head(10)


,Correlation,num of ratings
title,,
Ankara HiltonSA,1.000000,10
Bilkent Hotel and Conference Center,0.918966,9
Ostimpark Business Hotel,0.834675,7
CPAnkara Hotel,0.798652,7
Park 12,0.777295,8
Zarina Butik Hotel,0.759676,7
Aycha Butik Hotel,0.754893,9
Royal Anka Hotel Cankaya,0.748175,9
Otel KGZ Konagi,0.707243,8
